In [12]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
df = pd.read_csv('crawling\data\portal_news.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df.index.name='id'
# df.head()

In [61]:
sent = df['body'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)
sent = np.array(sent)

# 기사 제목을 이용해서 유사도 구하기

In [62]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
X_cv = cv.fit_transform(sent)

dim = X_cv.shape
print(dim)

(400, 32673)


In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer()
X_tv = tv.fit_transform(sent)
print(X_tv.shape)

(400, 32673)


In [64]:
from sklearn.metrics import pairwise
scikit_learn_cosine_sim = pairwise.cosine_similarity(X_cv)
scikit_learn_cosine_sim

array([[1.        , 0.02735542, 0.09727469, ..., 0.04234708, 0.040663  ,
        0.06812364],
       [0.02735542, 1.        , 0.08731375, ..., 0.03982077, 0.04540662,
        0.06199313],
       [0.09727469, 0.08731375, 1.        , ..., 0.09010967, 0.08343593,
        0.09886618],
       ...,
       [0.04234708, 0.03982077, 0.09010967, ..., 1.        , 0.12331741,
        0.1556804 ],
       [0.040663  , 0.04540662, 0.08343593, ..., 0.12331741, 1.        ,
        0.09675844],
       [0.06812364, 0.06199313, 0.09886618, ..., 0.1556804 , 0.09675844,
        1.        ]])

In [65]:
scikit_learn_cosine_sim.shape

(400, 400)

In [46]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject

In [47]:
!pip install --upgrade numpy

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 14.9/14.9 MB 4.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.2
    Uninstalling numpy-1.19.2:
      Successfully uninstalled numpy-1.19.2


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\SM-PC\\AppData\\Roaming\\Python\\Python38\\site-packages\\~umpy\\.libs\\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll'
Check the permissions.


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [49]:
!python.exe -m pip install --upgrade pip

Defaulting to user installation because normal site-packages is not writeable

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.



     ---------------------------------------- 2.1/2.1 MB 8.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1


In [52]:
import random
import re
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from numpy import sqrt, dot

random.seed(10)

doc1 = ["homelessness has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school work or vote for the matter"]

doc2 = ["it may have ends that do not tie together particularly well but it is still a compelling enough story to stick with"]

# 데이터를 불러오는 함수입니다.
def load_data(filepath):

    gensim_input = []
    with open(filepath, 'r') as f:
        for idx, line in enumerate(f):
            lowered_sent = line.rstrip()
            tagged_doc = TaggedDocument(lowered_sent, [idx])
            gensim_input.append(tagged_doc)
            
    return gensim_input
    
def cal_cosine_sim(v1, v2):
    # 벡터 간 코사인 유사도를 계산해 주는 함수를 완성합니다.
    top = dot(v1, v2)
    size1 = sqrt(dot(v1, v1))
    size2 = sqrt(dot(v2, v2))
    return top / (size1*size2)
    
# doc2vec 모델을 documents 리스트를 이용해 학습하세요.
documents = load_data("article.txt")
d2v_model = Doc2Vec(window=2, vector_size=50) # word2vec 클래스 설명 참고 (01 텍스트 전처리 및 단어 임베딩)
d2v_model.build_vocab(documents) # 수치형 인덱스 할당
# train(학습할 데이터, 문서개수, epochs)
d2v_model.train(documents, total_examples = d2v_model.corpus_count, epochs=5)



# 학습된 모델을 이용해 doc1과 doc2에 들어있는 문서의 임베딩 벡터를 생성하여 각각 변수 vector1과 vector2에 저장하세요.
vector1 = d2v_model.infer_vector(doc1)
vector2 = d2v_model.infer_vector(doc2)

# vector1과 vector2의 코사인 유사도를 변수 sim에 저장하세요.
sim = cal_cosine_sim(vector1, vector2)
# 계산한 코사인 유사도를 확인합니다.
print("sim", sim)

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 96 from C header, got 80 from PyObject